# SCC0270 - Redes Neurais e Aprendizado Profundo
---

Alunos:
- 10716504 - Helbert Moreira Pinto
- 10377708 - João Marcos Della Torre Divino

---

Exercício 3 - Dado o conjunto de dados: wine.dat:
- Utilizar uma rede MLP (utilizando o termo de momentum) para classificar o conjunto de dados.
- Utilizar uma RBF para classificar o mesmo conjunto.
- Comparar a acurácia dos modelos na classificação dos dados. 

---

### Introdução

Uma função de ativação de base radial é caracterizada por apresentar uma resposta que é monotonamente crescente ou decerescente com a distancia em relação a um ponto central.  
O centro e a taxa de variação (crescimento ou decrescimento) em cada direção são alguns dos parametros definidos que devem ser constantes.  

Temos funções de ativação da forma gaussiana, multiquadrativa ou chapeu mexicano. Utilizamos no exercicio a função de forma gaussiana, que possui a seguinte formula:
$$h_j = e^{ - \frac {(w-c_j)^2} {r^2_j}}$$

<figure>
    <img src='imgs/gaussiana.png' alt='Função gaussiana' width='1000' />
    <figcaption>Graficos da função gaussiana</figcaption>
</figure>

Neste caso, os elementos do vetor $\sigma_j = [\sigma_{j1}\ \sigma_{j2}\ \cdots \sigma_{jn}]^T$ são responsáveis pela taxa de decrescimento da gaussiana junto a cada coordenada do espaço de entrada, e o argumento da função exponencial é uma norma ponderada da diferença entre o vetor de entrada ($x$) e o centro da função de base radial ($c_j$).

---
### Implementação da Rede RBF (Radial Base Function)

As redes neurais com função de ativação de base radial (em inglês, radial-basis function, RBF) apresentam três diferenças principais em relação às redes MLP:
- Elas sempre apresentam uma única camada intermediária;
- Os neurônios de saída são sempre lineares;
- Os neurônios da camada intermediária têm uma função de base radial como função de ativação, ao invés de uma função sigmoidal. 

<figure>
    <img src='imgs/rede_rbf.png' alt='rede rbf' width='1000'/>
    <figcaption>Rede RBF para uma saída</figcaption>
</figure>

Ao invés da ativação interna de cada neurônio da camada intermediária se dar pelo emprego do produto escalar (produto interno) entre o vetor de entradas e o vetor de pesos, como no caso da rede MLP-BP, ela é obtida a partir de uma norma ponderada da diferença entre ambos os vetores.  


---
### Aplicação

---
#### Comparação com a Rede MLP-BP

In [1]:
from src.rbf import RBF
from src.mlp import MLP

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Dado um conjunto de dados, dividimos em treino/teste para a comparação de acuracia das redes no conjunto de testes
def exercicio(X, y, test_size:float=0.2, random_state:int=10, max_epocas:int=50, tol_amostra:float=0.2):
    X, y = np.array(X), np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, shuffle=True)

    # MLP
    rede_mlp = MLP(camadas=[len(X_train[0]), len(X_train[0]), len(y_train[0])],seed=random_state)
    rede_mlp.treino(X_train, y_train, taxa=0.5, alpha=0.5, max_epocas=max_epocas)
    acuracia_mlp = rede_mlp.teste(X_test, y_test, tol_amostra=tol_amostra)

    # RBF
    rede_rbf = RBF(n_classes=3,n_saidas=len(y_train[0]), seed=random_state)
    rede_rbf.treino(X_train, y_train, eta=0.5, max_epocas=max_epocas)
    acuracia_rbf = rede_rbf.teste(X_test, y_test, tol_amostra=tol_amostra)

    return acuracia_mlp, acuracia_rbf

---
#### Preparando os Dados

In [3]:
wine_df  = pd.read_csv('data/wine.data', header=None)
wine_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
# Normalização
scaler = MinMaxScaler()
wine_norm = scaler.fit_transform(wine_df)
wine_norm = pd.DataFrame(wine_norm)
wine_norm.columns = wine_df.columns
wine_norm.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.0,0.842105,0.191700,0.572193,0.257732,0.619565,0.627586,0.573840,0.283019,0.593060,0.372014,0.455285,0.970696,0.561341
1,0.0,0.571053,0.205534,0.417112,0.030928,0.326087,0.575862,0.510549,0.245283,0.274448,0.264505,0.463415,0.780220,0.550642
2,0.0,0.560526,0.320158,0.700535,0.412371,0.336957,0.627586,0.611814,0.320755,0.757098,0.375427,0.447154,0.695971,0.646933
3,0.0,0.878947,0.239130,0.609626,0.319588,0.467391,0.989655,0.664557,0.207547,0.558360,0.556314,0.308943,0.798535,0.857347
4,0.0,0.581579,0.365613,0.807487,0.536082,0.521739,0.627586,0.495781,0.490566,0.444795,0.259386,0.455285,0.608059,0.325963


In [5]:
# Separando as variavis explicativas da variavel resposta
wine_X = wine_norm.drop(columns=[0])
wine_y = wine_norm[[0]]

---
#### Comparação

In [6]:
test_size = 0.2
random_state = 123
max_epocas = 2
tol_amostra = 0.2

mlp, rbf = exercicio(
        wine_X, wine_y,
        test_size=test_size,
        random_state = random_state,
        max_epocas=max_epocas,
        tol_amostra = tol_amostra
    )

print('Acuracia MLP {:.2f}%'.format(100*mlp))
print('Acuracia RBF {:.2f}%'.format(100*rbf))

Acuracia MLP 41.67%
Acuracia RBF 94.44%


In [7]:
test_size = 0.2
random_state = 123
max_epocas = 25
tol_amostra = 0.2

mlp, rbf = exercicio(
        wine_X, wine_y,
        test_size=test_size,
        random_state = random_state,
        max_epocas=max_epocas,
        tol_amostra = tol_amostra
    )

print('Acuracia MLP {:.2f}%'.format(100*mlp))
print('Acuracia RBF {:.2f}%'.format(100*rbf))

Acuracia MLP 86.11%
Acuracia RBF 91.67%


In [8]:
test_size = 0.2
random_state = 10
max_epocas = 50
tol_amostra = 0.2

mlp, rbf = exercicio(
        wine_X, wine_y,
        test_size=test_size,
        random_state = random_state,
        max_epocas=max_epocas,
        tol_amostra = tol_amostra
    )

print('Acuracia MLP {:.2f}%'.format(100*mlp))
print('Acuracia RBF {:.2f}%'.format(100*rbf))

Acuracia MLP 88.89%
Acuracia RBF 91.67%


Analisamos agora quantas epocas de treino são necessárias para que a rede MLP-BP tenha mesma acurácia que a rede RBF

In [22]:
rnd = 15
tol_amostra = 0.2
X, y = np.array(wine_X), np.array(wine_y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rnd, shuffle=True)

In [23]:
rede_rbf = RBF(n_classes=3, n_saidas=len(y_train[0]), seed=rnd)
rede_rbf.treino(X_train, y_train, eta=0.5, max_epocas=1)
acuracia_rbf = rede_rbf.teste(X_test, y_test, tol_amostra=tol_amostra)
print('Acuracia da rede RBF com 1 epoca de treino: {:.2f}%'.format(100*acuracia_rbf))

Acuracia da rede RBF com 1 epoca de treino: 91.67%


In [24]:
rede_mlp = MLP(camadas=[len(X_train[0]), len(X_train[0]), len(y_train[0])],seed=rnd)
epoca = 1
acuracia_mlp = 0
while (epoca  < 500) and (acuracia_mlp < acuracia_rbf):
    rede_mlp.treino(X_train, y_train, taxa=0.3, alpha=0.7, max_epocas=1)
    acuracia_mlp = rede_mlp.teste(X_test, y_test, tol_amostra=tol_amostra)
    epoca += 1
print(f'Necessario {epoca} epocas para acuracia da rede MLP ser melhor ou igual a da rede RBF')

Necessario 66 epocas para acuracia da rede MLP ser melhor ou igual a da rede RBF
